In [1]:
import pandas as pd
import numpy as np

In [2]:
DATA_PATH_ORIG = r'../data/capture20110818.pcap.netflow.labeled.csv'
DATA_PATH_NEW = r'../data/scenario_10.csv'

In [3]:
column_names = ['_', '_2', 'duration', 'protocol', 'src_addr', '->', 'dst_addr', 'flags', 'tos', 'packets', 'bytes', 'flows', 'label']

df = pd.read_csv(
    DATA_PATH_ORIG,
    delimiter='\s+',
    skiprows=[0],
    header=None, 
    names=column_names,
    index_col=False,
    parse_dates={"start_date": [0,1]}
)
df = df.sort_values('start_date')
df.head()

,start_date,duration,protocol,src_addr,->,dst_addr,flags,tos,packets,bytes,flows,label
0,2011-08-18 10:19:13.328,0.002,TCP,147.32.86.166:33426,->,212.24.150.110:25443,FRPA_,0,4,321,1,Background
1,2011-08-18 10:19:13.328,4.995,UDP,82.39.2.249:41915,->,147.32.84.59:43087,INT,0,617,40095,1,Background
2,2011-08-18 10:19:13.329,4.996,UDP,147.32.84.59:43087,->,82.39.2.249:41915,INT,0,1290,1909200,1,Background
3,2011-08-18 10:19:13.330,0.000,TCP,147.32.86.166:42020,->,147.32.192.34:993,A_,0,1,66,1,Background
4,2011-08-18 10:19:13.330,0.000,TCP,212.24.150.110:25443,->,147.32.86.166:33426,FPA_,0,2,169,1,Background


In [4]:
portsrc = df["src_addr"].str.split(":", n = 1, expand = True) 
df['src_port'] = portsrc[1]
df["src_addr"] = portsrc[0]

portdest = df["dst_addr"].str.split(":", n = 1, expand = True) 
df['dst_port'] = portdest[1]
df["dst_addr"] = portdest[0]

df['src_port']=pd.to_numeric(df['src_port'], errors='coerce').fillna(-1).astype(np.int64)
df['dst_port']=pd.to_numeric(df['dst_port'], errors='coerce').fillna(-1).astype(np.int64)

In [5]:
df.drop(columns = ['->'], inplace=True)

In [6]:
df.head()

,start_date,duration,protocol,src_addr,dst_addr,flags,tos,packets,bytes,flows,label,src_port,dst_port
0,2011-08-18 10:19:13.328,0.002,TCP,147.32.86.166,212.24.150.110,FRPA_,0,4,321,1,Background,33426,25443
1,2011-08-18 10:19:13.328,4.995,UDP,82.39.2.249,147.32.84.59,INT,0,617,40095,1,Background,41915,43087
2,2011-08-18 10:19:13.329,4.996,UDP,147.32.84.59,82.39.2.249,INT,0,1290,1909200,1,Background,43087,41915
3,2011-08-18 10:19:13.330,0.000,TCP,147.32.86.166,147.32.192.34,A_,0,1,66,1,Background,42020,993
4,2011-08-18 10:19:13.330,0.000,TCP,212.24.150.110,147.32.86.166,FPA_,0,2,169,1,Background,25443,33426


In [7]:
df.isnull().values.any()

False

In [8]:
df.to_csv(DATA_PATH_NEW, index=False)